In [8]:
import sys
import os 

sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch")
sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/eval")

print(sys.path)

['/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/eval', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/eval', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch', '/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/analysis', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python38.zip', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/lib-dynload', '', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/site-packages', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_audio', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/site-packages/IPython/extensions', '/gpfs7kw/linkhome/rech/genini01/ucv88ce/.ipython']


In [9]:
import glob
import pickle
from tqdm import tqdm

In [10]:
import pyximport
pyximport.install()

import json
from os.path import exists, splitext, basename
from pathlib import Path

import torch
from cpc.criterion.clustering.clustering import loadClusterModule
import cpc.eval.eval_ABX

from cpc.eval.eval_ABX_clustering import ClusteringFeatures


from cpc.feature_loader import buildFeature, FeatureModule, loadModel
import argparse


In [11]:
#find a way to use jupyter notebook with gpu
#srun --ntasks=1 --cpus-per-task=10 --gres=gpu:1 --hint=nomultithread --time=01:00:00 --pty bash 

In [12]:
wav_fr = [x for x in glob.glob("/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr/*.wav")]
wav_en = [x for x in glob.glob("/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/en/*.wav")]

In [13]:
len(wav_fr)

7795

In [14]:
def get_units(kmeans_cp, wav_list, output_pkl=None):
    audio2units={}
    #kmeans_cp="/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/50h/00/kmeans50/checkpoint_last.pt"
    FeatureMaker = ClusteringFeatures(
                kmeans_cp,
                soft_clustering=False,
                encoder_layer=False,
                keepHidden=True,
                group_modes="onehot",
                onehot_dict=None,
    )
    for audio in tqdm(wav_list):
        audio_name = os.path.basename(audio)
        unit_list=[]
        T=FeatureMaker.feature_function(audio)
        a_tuple = (T == 1).nonzero(as_tuple=True)
        a=a_tuple[2]
        for i in range(len(a)):
            unit_list.append(a[i].item())
        audio2units[audio_name]=unit_list
    
    if output_pkl:
        with open(output_pkl, 'wb') as handle:
            pickle.dump(audio2units, handle)
    return audio2units

        

In [29]:
os.path.basename('/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0050.flac')
T

NameError: name 'T' is not defined

In [ ]:
#HERE IS CORRECT ( FR)



#exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/*/*/*/kmeans50")]
exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/*/*/kmeans50")]

prefix = "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/"
for exp  in exp_list:
    exp_path = exp[len(prefix):]
    print("EXP_PATH = ", exp_path)
    for lang in ["en", "fr"]:
        if lang == "fr":
            wav_list = wav_fr
        elif lang == "en":
            wav_list = wav_en
        kmeans_cp = os.path.join("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/", exp_path, "checkpoint_last.pt")
        output_path=os.path.join("/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/experiments/eval/inftrain", exp_path)
        output_pkl=os.path.join(output_path, lang+"_units.pkl")

        if not os.path.exists(output_path):
            os.makedirs(output_path)

        if not os.path.exists(output_pkl):
            get_units(kmeans_cp, wav_list, output_pkl=output_pkl)




EXP_PATH =  FR/1600h/01/kmeans50
EXP_PATH =  FR/1600h/00/kmeans50
EXP_PATH =  FR/800h/02/kmeans50
EXP_PATH =  FR/800h/01/kmeans50
EXP_PATH =  FR/800h/00/kmeans50
EXP_PATH =  FR/800h/03/kmeans50
EXP_PATH =  FR/400h/02/kmeans50
EXP_PATH =  FR/400h/05/kmeans50
EXP_PATH =  FR/400h/01/kmeans50
EXP_PATH =  FR/400h/04/kmeans50
EXP_PATH =  FR/400h/07/kmeans50
EXP_PATH =  FR/400h/00/kmeans50
EXP_PATH =  FR/400h/03/kmeans50
EXP_PATH =  FR/400h/06/kmeans50
EXP_PATH =  FR/3200h/00/kmeans50
EXP_PATH =  FR/200h/02/kmeans50
EXP_PATH =  FR/200h/05/kmeans50
EXP_PATH =  FR/200h/08/kmeans50
EXP_PATH =  FR/200h/11/kmeans50
EXP_PATH =  FR/200h/14/kmeans50
EXP_PATH =  FR/200h/01/kmeans50
EXP_PATH =  FR/200h/09/kmeans50
EXP_PATH =  FR/200h/04/kmeans50
EXP_PATH =  FR/200h/07/kmeans50
EXP_PATH =  FR/200h/13/kmeans50
EXP_PATH =  FR/200h/00/kmeans50
EXP_PATH =  FR/200h/03/kmeans50
EXP_PATH =  FR/200h/06/kmeans50
EXP_PATH =  FR/200h/12/kmeans50
EXP_PATH =  FR/200h/15/kmeans50
EXP_PATH =  FR/200h/10/kmeans50
EXP_P

  0%|          | 2/7795 [00:00<06:51, 18.95it/s]

Checkpoint loaded!

Loading ClusterModule at /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/50h/55/kmeans50/checkpoint_last.pt
Checkpoint loaded!



100%|██████████| 7795/7795 [06:56<00:00, 18.70it/s]


EXP_PATH =  FR/50h/50/kmeans50
Loading args from /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/50h/50/kmeans50/args.json

Clutering args:
{
    "DPLambda": 11,
    "DPMean": false,
    "MAX_ITER": 300,
    "batchSizeGPU": 200,
    "centroidLimits": null,
    "debug": false,
    "dimReduction": null,
    "dirOutput": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/50h/50/kmeans50",
    "encoder_layer": false,
    "extension": "wav",
    "getDistanceEstimation": false,
    "level_gru": 2,
    "load": null,
    "nClusters": 50,
    "nGroups": 1,
    "pathCheckpoint": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/50h/50/cpc_small/checkpoint_170.pt",
    "pathDB": "/gpfsssd/scratch/rech/cfs/commun/families/FR/50h/50",
    "perIterSize": 1406,
    "recursionLevel": 2,
    "save": true,
    "save_last": 5,
    "seqList": null,
    "sizeWindow": 10240,
    "train_mode": false
}
--------------------------------------------------
Loading checkpoint /gpfsssd/scratch/rech/cfs/com

...

100%|██████████| 7795/7795 [07:06<00:00, 18.26it/s]


EXP_PATH =  FR/50h/30/kmeans50
Loading args from /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/50h/30/kmeans50/args.json

Clutering args:
{
    "DPLambda": 11,
    "DPMean": false,
    "MAX_ITER": 300,
    "batchSizeGPU": 200,
    "centroidLimits": null,
    "debug": false,
    "dimReduction": null,
    "dirOutput": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/50h/30/kmeans50",
    "encoder_layer": false,
    "extension": "wav",
    "getDistanceEstimation": false,
    "level_gru": 2,
    "load": null,
    "nClusters": 50,
    "nGroups": 1,
    "pathCheckpoint": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/50h/30/cpc_small/checkpoint_195.pt",
    "pathDB": "/gpfsssd/scratch/rech/cfs/commun/families/FR/50h/30",
    "perIterSize": 1406,
    "recursionLevel": 2,
    "save": true,
    "save_last": 5,
    "seqList": null,
    "sizeWindow": 10240,
    "train_mode": false
}
--------------------------------------------------
Loading checkpoint /gpfsssd/scratch/rech/cfs/com

  0%|          | 2/7042 [00:00<06:32, 17.94it/s]

Checkpoint loaded!

Loading ClusterModule at /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/50h/30/kmeans50/checkpoint_last.pt
Checkpoint loaded!



 92%|█████████▏| 6469/7042 [06:17<00:40, 14.26it/s]

In [16]:
#HERE IS CORRECT ( FR)



#exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/*/*/*/kmeans50")]
exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/*/3200h/*/kmeans100")]
print(exp_list)

prefix = "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/"
for exp  in exp_list:
    exp_path = exp[len(prefix):]
    print("EXP_PATH = ", exp_path)
    for lang in ["en", "fr"]:
        if lang == "fr":
            wav_list = wav_fr
        elif lang == "en":
            wav_list = wav_en
        kmeans_cp = os.path.join("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/", exp_path, "checkpoint_last.pt")
        output_path=os.path.join("/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/experiments/eval/inftrain", exp_path)
        output_pkl=os.path.join(output_path, lang+"_units.pkl")

        if not os.path.exists(output_path):
            os.makedirs(output_path)

        if not os.path.exists(output_pkl):
            get_units(kmeans_cp, wav_list, output_pkl=output_pkl)




['/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans100']
EXP_PATH =  EN/3200h/00/kmeans100
Loading args from /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans100/args.json

Clutering args:
{
    "DPLambda": 11,
    "DPMean": false,
    "MAX_ITER": 300,
    "batchSizeGPU": 200,
    "centroidLimits": null,
    "debug": false,
    "dimReduction": null,
    "dirOutput": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans100",
    "encoder_layer": false,
    "extension": "wav",
    "getDistanceEstimation": false,
    "level_gru": 2,
    "load": null,
    "nClusters": 100,
    "nGroups": 1,
    "pathCheckpoint": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/cpc_small/checkpoint_25.pt",
    "pathDB": "/gpfsssd/scratch/rech/cfs/commun/families/EN/3200h/00",
    "perIterSize": 1406,
    "recursionLevel": 2,
    "save": true,
    "save_last": 5,
    "seqList": null,
    "sizeWindow": 10240,
    "train_mode": false
}
-----------

  0%|          | 2/7042 [00:00<07:53, 14.86it/s]

Checkpoint loaded!

Loading ClusterModule at /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans100/checkpoint_last.pt
Checkpoint loaded!



100%|██████████| 7042/7042 [03:07<00:00, 37.52it/s]


Loading args from /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans100/args.json

Clutering args:
{
    "DPLambda": 11,
    "DPMean": false,
    "MAX_ITER": 300,
    "batchSizeGPU": 200,
    "centroidLimits": null,
    "debug": false,
    "dimReduction": null,
    "dirOutput": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans100",
    "encoder_layer": false,
    "extension": "wav",
    "getDistanceEstimation": false,
    "level_gru": 2,
    "load": null,
    "nClusters": 100,
    "nGroups": 1,
    "pathCheckpoint": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/cpc_small/checkpoint_25.pt",
    "pathDB": "/gpfsssd/scratch/rech/cfs/commun/families/EN/3200h/00",
    "perIterSize": 1406,
    "recursionLevel": 2,
    "save": true,
    "save_last": 5,
    "seqList": null,
    "sizeWindow": 10240,
    "train_mode": false
}
--------------------------------------------------
Loading checkpoint /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/E

  0%|          | 6/7795 [00:00<02:44, 47.21it/s]

Checkpoint loaded!

Loading ClusterModule at /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans100/checkpoint_last.pt
Checkpoint loaded!



100%|██████████| 7795/7795 [03:07<00:00, 41.61it/s]


In [37]:
#/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean

#HERE IS CORRECT ( FR)


wav_en_LS_clean = [x for x in glob.glob("/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/*/*/*.wav")]

exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans50")]
lang="en"
prefix = "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/"
for exp  in exp_list:
    exp_path = exp[len(prefix):]
    print("EXP_PATH = ", exp_path)
    wav_list = wav_en_LS_clean
    kmeans_cp = os.path.join("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/", exp_path, "checkpoint_last.pt")
    output_path=os.path.join("/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/experiments/eval/inftrain", exp_path)
    output_pkl=os.path.join(output_path, lang+"_units_LS_clean.pkl")
    print(output_pkl)

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    if not os.path.exists(output_pkl):
        print("here")
        get_units(kmeans_cp, wav_list, output_pkl=output_pkl)




EXP_PATH =  EN/3200h/00/kmeans50
/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/experiments/eval/inftrain/EN/3200h/00/kmeans50/en_units_LS_clean.pkl
here
Loading args from /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans50/args.json

Clutering args:
{
    "DPLambda": 11,
    "DPMean": false,
    "MAX_ITER": 300,
    "batchSizeGPU": 200,
    "centroidLimits": null,
    "debug": false,
    "dimReduction": null,
    "dirOutput": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans50",
    "encoder_layer": false,
    "extension": "wav",
    "getDistanceEstimation": false,
    "level_gru": 2,
    "load": null,
    "nClusters": 50,
    "nGroups": 1,
    "pathCheckpoint": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/cpc_small/checkpoint_25.pt",
    "pathDB": "/gpfsssd/scratch/rech/cfs/commun/families/EN/3200h/00",
    "perIterSize": 1406,
    "recursionLevel": 2,
    "save": true,
    "save_last": 5,
    "seqList": null,
    

  0%|          | 5/2620 [00:00<00:56, 46.00it/s]

Checkpoint loaded!

Loading ClusterModule at /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans50/checkpoint_last.pt
Checkpoint loaded!



100%|██████████| 2620/2620 [01:38<00:00, 26.60it/s]


In [30]:
wav_en_LS_clean = [x for x in glob.glob("/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/*/*/*.wav")]
wav_en_LS_clean 

['/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0049.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0039.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0037.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0062.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0056.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0032.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0000.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0002.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122797-0063.flac.wav',
 '/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/672/122797/672-122

In [ ]:
#need to double check. Function that is not thoroughly checked but seems to be working
#this should lalow us to get average length

#should obviously be done on a lit of file. 


units={x:[] for x in range(50)}
T=FeatureMaker.feature_function(audio_file)
a_tuple = (T == 1).nonzero(as_tuple=True)
a=a_tuple[2]
curr_unit=0
for i in range(len(a)):
    curr_unit+=1
    if i+1 == len(a):#edge case
        units[a[i].item()].append(curr_unit)
        curr_unit=0
    elif a[i+1].item() != a[i].item():
        units[a[i].item()].append(curr_unit)
        curr_unit=0

        